In [11]:
import os
import json
import requests
import random

from django.core.files.base import ContentFile
from shop.models import Category, Shop, Item
from urllib.parse import urljoin
from fake_useragent import UserAgent

In [12]:
class food_date:
    HOST = 'https://www.yogiyo.co.kr'
    
    def __init__(self, headers):
        self.headers = headers
        
    def get_restaurant_list_by_geo(self, zip_code):
        url = self.HOST + '/api/v1/restaurants-geo/'
        
        params = dict(
            items=20,
            lat=37.5495123860853,
            lng=127.047075859646,
            order='rank',
            page=0,
            search='',
            zip_code=zip_code,
        )
        
        res = requests.get(url, params=params, headers=self.headers)
        return res.json()
    
    def get_menu_list(self, restaurant_id):
        base_url = self.HOST + '/api/v1/restaurants/{restaurant_id}/menu/?add_photo_menu=original'
        url = base_url.format(restaurant_id=restaurant_id)
        res = requests.get(url, headers=self.headers)
        return res.json()

In [13]:
user_agent = UserAgent()

In [14]:
headers = {
    'x-apikey': 'iphoneap',
    'x-apisecret': 'fe5183cc3dea12bd0ce299cf110a75a2',
    'User_Agent': user_agent.Chrome,
}

In [15]:
headers

{'x-apikey': 'iphoneap',
 'x-apisecret': 'fe5183cc3dea12bd0ce299cf110a75a2',
 'User_Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1468.0 Safari/537.36'}

In [16]:
food = food_date(headers)

In [17]:
zip_code = '143201'
restaurants = food.get_restaurant_list_by_geo('143201')['restaurants']
restaurants

[{'rating': 200037.800431896,
  'subtitle': '',
  'additional_discount_currently_active': False,
  'free_delivery_threshold': 0,
  'delivery_fee_explanation': '배달 요금 2500원, 최소 주문 금액 9000원',
  'is_available_delivery': True,
  'threshold': 9000,
  'adv_distance': None,
  'open': True,
  'discount_until': None,
  'central_billing': False,
  'city': '서울',
  'review_reply_count': 1543,
  'sms_bonus': True,
  'franchise_id': None,
  'phone_downlisted': False,
  'new_mark_date': None,
  'franchise_name': None,
  'review_image_count': 1050,
  'top100_restaurant': False,
  'is_available_pickup': False,
  'representative_menus': '베테랑 국물떡볶이, 베테랑 라볶이',
  'additional_discount_per_menu': 0,
  'delivery_fee': 2500,
  'additional_discount_pickup': 0,
  'phone': '050712936352',
  'additional_discount_only_for_user': False,
  'logo_curation': None,
  'has_terminal': True,
  'estimated_delivery_time_key': 60,
  'categories': ['한식', '1인분주문', '분식', '야식'],
  'min_order_amount': 9000,
  'distance': 3.4626037

In [18]:
for rest in restaurants:
    category_name = rest['categories'][0]
    print(category_name)
    category = Category.objects.get(name=category_name)
    latlng = '{lat},{lng}'.format(**rest)
    logo_url = urljoin(food_date.HOST, rest['logo_url'])
    shop = Shop(category=category, name=rest['name'], latlng=latlng, meta=rest)
    
    logo_name = os.path.basename(logo_url)
    logo_data = requests.get(logo_url).content
    shop.photo.save(logo_name, ContentFile(logo_data), save=False)
    print('{name}: {lat},{lng}, {categories}, {logo_url}'.format(**rest))
    
    shop.save()

한식
베테랑떡볶이: 37.5344882628312,127.081475610118, ['한식', '1인분주문', '분식', '야식'], /media/restaurant_logos/베테랑떡볶이_20190402_가맹로고이미지약정서_crop_200x200.jpeg
프랜차이즈
빅스타피자-서울건대점: 37.5377246470307,127.081528503217, ['프랜차이즈', '피자양식'], /media/franchise_logos/빅스타피자_20180118_Franchise이미지약정서_crop_200x200_l1CqNyp.jpg
치킨
또래오래-성동성수역점: 37.5451976360753,127.0550942422, ['치킨', '프랜차이즈'], /media/franchise_logos/또래오래_20151126_Franchise이미지약정서_crop_200x200_89SjZAD.jpg
1인분주문
맥도날드-한양대점: 37.5572612448551,127.041909296569, ['1인분주문', '피자양식', '프랜차이즈'], /media/franchise_logos/맥도날드_20190523_Franchise_crop_200x200.png
프랜차이즈
KFC-세종대점: 37.5472166426971,127.073569298591, ['프랜차이즈', '치킨', '피자양식'], /media/franchise_logos/KFC_20181218_Franchise이미지약정서_crop_200x200.jpg
야식
금룡-한양대점: 37.5595191306642,127.040535908241, ['야식', '1인분주문', '중식'], /media/restaurant_logos/반반탕짜면01_20131128_FoodAD_crop_200x200_j3E9t7A.jpg
야식
양치는아저씨-건대직영점: 37.5416715114726,127.070352976018, ['야식', '한식', '피자양식', '중식'], /media/restaurant_logos/업체자체_20190812_354501_양치는

In [19]:
for shop in Shop.objects.all():
    print(shop.name)

빅스타피자-서울건대점
또래오래-광진광장점
롯데리아-구의역점
교촌치킨-구의2호점
빙달DessertCafe-건대점
불쭈꾸미&소곱창
장수장충동왕족발보쌈
배달의쌀국수-건대점
후라이드참잘하는집-구의점
요리왕
직화쭈꾸미&소곱창
소소떡볶이-자양직영점
벌떡-자양점
반올림피자샵-자양성수점
뿔발불막창불닭발-광진구점
잭아저씨족발보쌈-구의점
롯데리아-건대역점
자가제빵선명희피자-구의점
베테랑떡볶이
대마족발-구의점
인정국물떡볶이-성수화양점
군자소곱창쭈꾸미
굽네치킨&피자-구의1동점
리얼피자-건대점
멕시카나-성수점
페리카나-광장동점
치킨선생-건대점
피자119(구의점)
달려라달봉이떡볶이-광진점
BHC-광나루점
복만루
마라판다-자양점
꿀마니닭강정-광진점
꼽사리
바른치킨-건대역2호점
미림장충왕족발보쌈
밥장인돼지찌개-자양점
추억닭발-군자점
푸른바다찜앤탕-자양점
허니안심탕수육
스시도쿠-본점
물고기반찬by생선구이
대구안지랑곱창막창
치즈시즈닝닭강정
59쌀피자-구의역점
육회한연어-건대점
한명숙피자치킨-광진점
카페다비다-핸드드립&더치전문
BHC-구의역점
참살이임실치즈피자
페리카나-구의점
밀방떡-광진점
쪽갈비에미치닭-본점
미식가의1人닭도리1人찜닭
왕십리닭내장탕
황금손가락초밥
본스치킨-구의본점
진화쿵푸마라탕-구의점
청두인상
뿅가야채곱창-광진점
또래오래-자양점
윤호네양꼬치&중식요리
어쩌다마주친족발
족발야시장-건대입구역점
의정부정통부대찌개
마당족발
야한곱창-건대점
쁘이치킨
멕시카나-아차산역점
헬로피자-건대입구점
구의동대양참치
가마로강정-자양점
밥통령-건대점
정성찬-건대점
고기한끼-건대점
아빠는요리사
짚신매운갈비찜-건대점
굽네치킨&피자-광장점
맛짱곱창
매운녀석들
윙스치킨
신강투루판양꼬치
국민닭발-광진구점
맛닭꼬-어린이대공원역점
피자연합-화양점
옛날종로빈대떡-화양점
명품도시락갓토리
잇쏘핫갈비찜
참존숯불바베큐치킨
투다리-대청점
눈물나는떡볶이앤눈꽃빙수-광진점
백가네닭요리끝판왕탕찜야식
곱창은요리다-구의점
곱창고-구의점
땡초꼼장어&닭발
웁스떡볶이-중곡점
오성족발-구의점
돈까스일팔공
건대마당족발
여기야싼티포
육집으로-구의점

In [ ]:
for shop in Shop.objects.all():
    restaurant_id = shop.meta['id']
    menu_list = food.get_menu_list(restaurant_id)
    print(shop.name)
    for sub_menu_list in menu_list:
        items = sub_menu_list['items']
        for item_meta in items:
            item = Item(shop=shop, name=item_meta['name'], amount=item_meta['price'])
            
            item_image_url = item_meta.get('image', '')
            if item_image_url:
                item_image_url = urljoin(food_date.HOST, item_meta['image'])
                
                item_image_name = os.path.basename(item_image_url)
                item_image_data = requests.get(item_image_url).content
                item.photo.save(item_image_name, ContentFile(item_image_data), save=False)
                
            print('saving item : {}'.format(item.name))
            item.save()

빅스타피자-서울건대점
saving item : 흐르쥬퐁듀 시카고피자（10인치）
saving item : 리턴오리지널 피자
saving item : 갈릭새우스테이크 피자
saving item : 폭탄 피자
saving item : 직화바베큐 피자
saving item : 치폴레 쉬림프 디트로이트피자
saving item : 반반피자 16,900메뉴
saving item : 페퍼로니 시카고피자¸
saving item : 반반피자 17,900메뉴
saving item : 빅스타스파게티 세트A¸
saving item : 폭탄 피자
saving item : 빅스타스파게티 세트B
saving item : 스위트크림&콘슈 시카고피자¸
saving item : 콤비네이션 피자
saving item : 치즈 시카고피자¸
saving item : 흐르쥬퐁듀 시카고피자（10인치）
saving item : 치폴레 쉬림프 디트로이트피자
saving item : 8치즈소스 & 그릴드비프 디트로이트피자
saving item : 갈릭새우스테이크 피자
saving item : 폭탄 피자
saving item : 콰트로프리 피자
saving item : 치즈타코 피자
saving item : 직화바베큐 피자
saving item : 치폴레 쉬림프 디트로이트피자
saving item : 8치즈소스 & 그릴드비프 디트로이트피자
saving item : 참피온피자
saving item : 흐르쥬퐁듀 시카고피자（10인치）
saving item : 스위트크림&콘슈 시카고피자（10인치）¸
saving item : 리턴오리지널 피자
saving item : 치폴레 쉬림프 디트로이트피자
saving item : 8치즈소스 & 그릴드비프 디트로이트피자
saving item : 빅스타스파게티 세트A¸
saving item : 빅스타스파게티 세트B
saving item : 빅스타 스파게티 세트C
saving item : 콰트로프리 피자
saving item : 콰트로 피자
saving item : 반반피자 23

불쭈꾸미&소곱창
saving item : 소곱창구이
saving item : 모듬구이
saving item : 세트（쭈꾸미＋계란찜＋주먹밥）
saving item : 세트（쭈삼＋계란찜＋주먹밥）
saving item : 세트（쭈곱＋계란찜＋주먹밥）
saving item : 세트（쭈새＋계란찜＋주먹밥）
saving item : 세트（쭈곱새＋계란찜＋주먹밥）
saving item : 소곱창구이 200g
saving item : 소막창구이 200g
saving item : 소염통구이 200g
saving item : 직화쭈꾸미볶음
saving item : 직화쭈삼볶음
saving item : 직화쭈곱볶음
saving item : 직화쭈새볶음
saving item : 직화제육볶음
saving item : 직화쭈새볶음
saving item : 직화쭈꾸미볶음
saving item : 직화쭈삼볶음
saving item : 직화쭈곱볶음（소곱창）
saving item : 스페셜직화쭈곱새볶음
saving item : 제육볶음
saving item : 계란찜
saving item : 주먹밥
saving item : 공기밥
saving item : 스프라이트 355mL
saving item : 콜라 355mL
saving item : 환타 355mL
장수장충동왕족발보쌈
saving item : 감자탕 中
saving item : 매운숯불족발 中
saving item : 보족세트 中
saving item : 장수족발 中
saving item : 웰빙굽네족발 中
saving item : 보쌈 中
saving item : 장수족발 小
saving item : 장수족발 中
saving item : 알뜰보쌈 小（400g）
saving item : 보쌈 小
saving item : 보쌈 中
saving item : 장수족발 大
saving item : 보족세트 小
saving item : 살로만족발  小（400g）
saving item : 반반세트 小
saving item : 보족세트 中
saving

saving item : 주먹밥
saving item : 공기밥
saving item : 스프라이트 355mL
saving item : 콜라 355mL
saving item : 환타 355mL
소소떡볶이-자양직영점
saving item : 소소떡볶이 
saving item : 소매떡볶이 
saving item : 파스타떡볶이
saving item : 소소짜장떡볶이
saving item : 순대떡볶음
saving item : 소소마라닭볶음탕
saving item : 소소떡볶이 
saving item : 소소라볶이 
saving item : 스페셜튀김（김말이＋오징어＋새우）
saving item : 소매떡볶이 
saving item : 파스타떡볶이
saving item : 소소짜장떡볶이
saving item : 인기메뉴 세트1（소매떡볶이（매운）＋모듬튀김＋스팸치즈김밥＋쿨피스）
saving item : 자신있는 순대M（내장 포함）
saving item : 인기메뉴 세트2（파스타떡볶이（매콤）＋모듬튀김＋스팸치즈김밥＋쿨피스）
saving item : 순대떡볶음
saving item : 소소마라닭볶음탕
saving item : 소소김밥
saving item : 인기메뉴 세트1（소매떡볶이（매운）＋모듬튀김＋스팸치즈김밥＋쿨피스）
saving item : 인기메뉴 세트2（파스타떡볶이（매콤）＋모듬튀김＋스팸치즈김밥＋쿨피스）
saving item : 패밀리SET（4인）（소매떡볶이M＋파스타떡볶이＋모듬튀김＋스팸치즈김밥＋순대L＋쿨피스）
saving item : 소소떡볶이 
saving item : 소매떡볶이 
saving item : 파스타떡볶이
saving item : 소소짜장떡볶이
saving item : 소소라볶이 
saving item : 순대떡볶음
saving item : 수제모듬튀김
saving item : NEW-베스트튀김（크랩＋오징어＋고구마＋고추） 
saving item : 스페셜튀김（김말이＋오징어＋새우）
saving item : 야채튀김（고구마＋단호박＋양파＋마늘쫑）
saving

saving item : 공기밥
saving item : 마늘소스
saving item : 불보쌈소스
saving item : 캔 콜라
saving item : 캔 사이다
saving item : 캔 웰치스
saving item : 캔 스파클링 사과
saving item : 콜라500mL
saving item : 사이다500mL
saving item : 콜라1.25L
saving item : 사이다1.25L
롯데리아-건대역점
saving item : 와규에디션Ⅱ세트
saving item : DoubleX2세트
saving item : NEW홈패밀리팩
saving item : NEW홈더블팩
saving item : 와규에디션Ⅱ
saving item : 한우불고기버거
saving item : 불고기버거세트
saving item : 새우버거세트
saving item : DoubleX2세트
saving item : 핫크리스피버거세트
saving item : 새우버거
saving item : 와규에디션Ⅱ세트
saving item : T-REX버거 세트
saving item : 모짜렐라인더 버거-베이컨 세트
saving item : 치킨 1조각
saving item : 지파이 고소한맛（S）
saving item : 와규에디션Ⅱ
saving item : DoubleX2버거
saving item : T-REX버거
saving item : 모짜렐라인더버거-해쉬
saving item : 모짜렐라인더버거-더블
saving item : 모짜렐라인더버거-베이컨
saving item : AZ오리지널
saving item : 데리버거
saving item : 치킨버거
saving item : 불고기버거
saving item : 새우버거
saving item : 원조빅불
saving item : 한우불고기버거
saving item : 핫크리스피버거
saving item : 클래식치즈버거
saving item : 와규에디션Ⅱ세트
saving item : DoubleX2세트
saving it

saving item : 국물떡볶이（2인）
saving item : 모듬튀김（오징어1＋고구마1＋잡채말이1＋야끼2＋군만두1＋김말이2）
saving item : 인정알뜰세트1（국물떡볶이2인＋부산꼬지어묵3개＋인정김밥5줄＋쿨피스450mL）
saving item : 수제오징어튀김
saving item : 스팸＋참치마요컵밥
saving item : 후룩떡볶이（2인）
saving item : 국물떡볶이（1인）
saving item : 국물떡볶이（2인）
saving item : 국물떡볶이（4인）
saving item : 국물떡볶이（오뎅많이4인）
saving item : 깻잎순대떡볶이
saving item : 깻잎순대떡볶이（2~3인）
saving item : 눈꽃치즈 돈까스 떡볶이
saving item : 눈꽃치즈 돈까스 떡볶이（2~3인） 
saving item : 후룩떡볶이（1인）
saving item : 후룩떡볶이（2인）
saving item : 후룩떡볶이（3~4인）
saving item : 반반세트
saving item : 인정알뜰세트1（국물떡볶이2인＋부산꼬지어묵3개＋인정김밥5줄＋쿨피스450mL）
saving item : 인정알뜰세트2（국물떡볶이2인＋수제모듬튀김＋찰순대＋쿨피스450mL）
saving item : 인정알뜰세트3（국물떡볶이2인＋순살치킨＋콜라500mLor쿨피스450mL）
saving item : 인정알뜰세트4（국물떡볶이2인＋주먹밥＋부산꼬지어묵3개＋찰순대＋쿨피스450mL）
saving item : 인정알뜰세트5（국물떡볶이2인＋주먹밥＋수제모듬튀김＋부산꼬지어묵3개＋찰순대＋쿨피스450mL）
saving item : 인정알뜰세트6（국물떡볶이1인＋주먹밥＋숯불무뼈닭발＋쿨피스 450mL）
saving item : 꼬치소스
saving item : 수제오징어튀김
saving item : 고추튀김（1개）
saving item : 새우튀김 （1개）
saving item : 야끼만두 （3개）
saving item : 피카츄 돈까스 
saving item : 튀김오뎅（6개）
savi

...

saving item : 레피시에 사과쨈（370g）
가장매력적인족발-본점
saving item : 족발（중）
saving item : 알뜰실속족발
saving item : 알뜰실속불족발
saving item : 알뜰실속보쌈
saving item : 족발（소）
saving item : 족발（대）
saving item : 알뜰실속족발
saving item : 족발（대）
saving item : 불족보쌈 （ 대 ）
saving item : 족발（소）
saving item : 족발（중）
saving item : C세트 （오리지날족발小＋보쌈小）
saving item : 반반족발 （대） （매운맛）
saving item : 불족보쌈 （ 중 ）
saving item : 알뜰실속족발
saving item : 알뜰실속불족발
saving item : 알뜰실속보쌈
saving item : A세트 （오리지날족발大＋보쌈大）
saving item : B세트 （오리지날족발中＋보쌈中）
saving item : C세트 （오리지날족발小＋보쌈小）
saving item : 족발（소）
saving item : 족발（중）
saving item : 족발（대）
saving item : 직화불족발 （소） 
saving item : 직화불족발 （중）
saving item : 직화불족발 （대）
saving item : 반반족발 （소）（매운맛）
saving item : 반반족발 （중）（매운맛）
saving item : 반반족발 （대） （매운맛）
saving item : 마늘족발 （대）
saving item : 마늘족발 （중）
saving item : 마늘족발 （소）
saving item : 불족보쌈 （소）
saving item : 불족보쌈 （ 중 ）
saving item : 불족보쌈 （ 대 ）
saving item : 족발보쌈（소）
saving item : 족발보쌈 （중）
saving item : 족발보쌈 （대）
saving item : 보쌈 （ 소）
saving item : 보쌈 （ 중 ）
saving ite

saving item : 유부초밥 4p
saving item : 새우튀김 3p
saving item : 치즈스틱 5p
saving item : 왕만두 5p
saving item : 매콤김말이 7p
saving item : 매운군만두 5p
saving item : 야끼만두 5p
saving item : 고구마맛탕 12p
saving item : 치즈 추가
saving item : 계란후라이
saving item : 소우동 추가
saving item : 공기밥 추가
saving item : 각종소스
saving item : 콜라 캔
saving item : 콜라 1.25mL
saving item : 사이다 캔
saving item : 사이다 1.25mL
saving item : 생수 2L
정다믄김밥-자양점
saving item : 정다믄김밥
saving item : 치즈김밥
saving item : 참치김밥
saving item : 돈까스김밥
saving item : 매콤힐링낙지덮밥
saving item : 매콤힐링소고기덮밥
saving item : 정다믄김밥
saving item : 참치김치찌개
saving item : 돈까스＋쫄면
saving item : 참치김밥
saving item : 소고기야채죽
saving item : 전복죽
saving item : 쫄면
saving item : 치즈돈까스
saving item : 제육직화덮밥
saving item : 새우볶음밥
saving item : 정다믄김밥
saving item : 치즈김밥
saving item : 땡초김밥
saving item : 김치김밥
saving item : 참치김밥
saving item : 돈까스김밥
saving item : 직화소불고기김밥
saving item : 떡갈비김밥
saving item : 새우튀김김밥
saving item : 스파이시치킨김밥
saving item : 킹소시지김밥
saving item : 날치알김밥
saving item : 매콤힐링낙지덮밥
saving item 

saving item : 황태채고추장조림
saving item : 고추맛절임
saving item : 무말랭이
saving item : 작은양념고추
saving item : 깐마늘장아찌（초절임）
saving item : 파래돌김볶음 65g
saving item : 건새우땅콩조림
saving item : 간장가문어조림
saving item : 땅콩조림
saving item : 백김치
saving item : 총무깍뚜기
saving item : 갓김치
saving item : 무생채
saving item : 총각김치
saving item : 나박물김치（무＋배추＋배＋당근＋고추가루살짝）
saving item : 깻잎김치
saving item : 열무김치
saving item : 단호박샐러드
saving item : 감자샐러드
saving item : 살사후실리샐러드（파스타）
saving item : 미역줄기샐러드
saving item : 고구마샐러드
saving item : 오징어젓
saving item : 비빔오징어젓（다짐오징어젓）
saving item : 명란젓
saving item : 양념순살닭강정
saving item : 도톰등심돈까스（2장）
saving item : 순살치킨텐더（치킨안심살）7~9조각
saving item : 양념탕수육
saving item : 닭봉＋핫윙（12p）
saving item : 고구마맛탕
saving item : 새우튀김（5마리）
saving item : 간장순살닭강정
saving item : 간장양념탕수육
saving item : 생선까스（3장）
saving item : 김부각
saving item : 연근칩
saving item : 찹쌀미역부각
saving item : 우엉칩
saving item : 찹쌀다시마부각
saving item : 고추부각
saving item : 식혜 1L
saving item : 누룽지 280g
saving item : 호박죽 500g（냉장포장）
saving item : 한우소고기죽 500g （냉장포장

saving item : 에그핫도그
saving item : 바베큐핫도그세트
saving item : 허니핫도그세트
saving item : 스파이시핫도그세트
saving item : 치즈핫도그세트
saving item : 그릴치킨핫도그세트
saving item : 어니언핫도그세트
saving item : 게살핫도그세트
saving item : 땡초핫도그세트
saving item : 할라피뇨핫도그세트
saving item : 피자핫도그세트
saving item : 갈릭핫도그세트
saving item : 스위트칠리핫도그세트
saving item : 머쉬룸핫도그세트
saving item : 에그핫도그세트
saving item : 불고기버거
saving item : 원치즈버거
saving item : 스파이시버거
saving item : 치즈불고기버거
saving item : 게살버거
saving item : 모짜렐라버거
saving item : 스위트칠리버거
saving item : 파인버거
saving item : 베이컨토마토버거
saving item : 더블치즈버거 
saving item : 갈릭머쉬룸버거
saving item : 인크레더블버거
saving item : 불고기버거세트
saving item : 원치즈버거세트
saving item : 스파이시버거세트
saving item : 치즈불고기버거세트
saving item : 게살버거세트
saving item : 모짜렐라버거세트
saving item : 스위트칠리버거세트
saving item : 파인버거세트
saving item : 베이컨토마토버거세트
saving item : 더블치즈버거세트
saving item : 갈릭머쉬룸버거세트
saving item : 인크레더블버거세트
saving item : 꿉자버거
saving item : 치즈버거
saving item : 베이컨치즈버거
saving item : 땡초버거
saving item : 꿉자버거세트
saving item : 치즈버거세트
saving ite

saving item : 후라이드＋6초치킨
saving item : 후라이드＋장스치킨
saving item : 후라이드 순살＋양념치킨 순살
saving item : 후라이드 순살＋간지치킨 순살
saving item : 후라이드 순살＋고추치킨 순살
saving item : 후라이드 순살＋6초치킨 순살
saving item : 후라이드 순살＋장스치킨 순살
saving item : 봉 15조각
saving item : 윙 15조각
saving item : 똥집튀김＋닭봉15조각
saving item : [추천]간지윙봉세트25조각
saving item : [추천]고추윙봉세트25조각
saving item : 쫀득치즈볼（5개）
saving item : 콘치즈볼（5개）
saving item : 똥집튀김
saving item : 웨지감자
saving item : 진미튀김
saving item : 콜라 1.25L
saving item : 콜라 1.25L
saving item : 사이다 1.5L
saving item : 매운양념소스
saving item : 마요네즈소스
saving item : 치킨 무
구름산추어탕&족발보쌈
saving item : 추어탕
saving item : 통추어탕
saving item : 전복추어탕
saving item : 뚝배기불고기
saving item : 수제돈까스
saving item : 치즈돈까스
saving item : 족발
saving item : 매운족발
saving item : 반반족발（족발＋매운족발）
saving item : 쟁반막국수
saving item : 간재미무침
saving item : 보쌈
saving item : 추어튀김
나는밥조폭이다-광진구점
saving item : 양념게장 300g
saving item : 공기밥
saving item : 캔콜라 355mL
saving item : 이대장 양념게장 1人 스페셜세트（양념게장300g＋반찬 3종 中 택1＋도토리묵냉국1ea＋후식누룽지탕1ea＋계란밥1ea）
saving item :